In [2]:
import kaggle 
!kaggle datasets download -d taeefnajib/used-car-price-prediction-dataset

Dataset URL: https://www.kaggle.com/datasets/taeefnajib/used-car-price-prediction-dataset
License(s): Attribution 4.0 International (CC BY 4.0)




  0%|          | 0.00/109k [00:00<?, ?B/s]
100%|██████████| 109k/109k [00:00<00:00, 160kB/s]
100%|██████████| 109k/109k [00:00<00:00, 160kB/s]


In [3]:
import zipfile
zip = zipfile.ZipFile('used-car-price-prediction-dataset.zip')
zip.extractall()
zip.close()

# Import Lib

In [222]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [223]:
df = pd.read_csv('used_cars.csv')

**Brand & Model:** Identify the brand or company name along with the specific model of each vehicle.  

**Model Year:** Discover the manufacturing year of the vehicles, crucial for assessing depreciation and technology advancements.  

**Mileage:** Obtain the mileage of each vehicle, a key indicator of wear and tear and potential maintenance requirements.  

**Fuel Type:** Learn about the type of fuel the vehicles run on, whether it's gasoline, diesel, electric, or hybrid.  

**Engine Type:** Understand the engine specifications, shedding light on performance and efficiency.  

**Transmission:** Determine the transmission type, whether automatic, manual, or another variant.  

**Exterior & Interior Colors:** Explore the aesthetic aspects of the vehicles, including exterior and interior color options.  

**Accident History:** Discover whether a vehicle has a prior history of accidents or damage, crucial for informed decision-making.  

**Clean Title:** Evaluate the availability of a clean title, which can impact the vehicle's resale value and legal status.  

**Price:** Access the listed prices for each vehicle, aiding in price comparison and budgeting.  

In [224]:
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


In [225]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


# Data Cleaning

In [226]:
df['clean_title'].unique()

array(['Yes', nan], dtype=object)

In [227]:
def count_dash(df):
    return (df == '–').sum()

dash_counts =count_dash(df)

cols_with_dash = dash_counts[dash_counts > 0].index
df[cols_with_dash] = df[cols_with_dash].replace('–', np.nan)

In [228]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3794 non-null   object
 5   engine        3964 non-null   object
 6   transmission  4005 non-null   object
 7   ext_col       3994 non-null   object
 8   int_col       3876 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


In [229]:
df.sample(5)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
3355,Ford,Edge SEL,2021,"17,836 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Silver,Ebony,None reported,Yes,"$31,933"
2371,Mazda,Mazda3 s Sport,2008,"77,700 mi.",Gasoline,156.0HP 2.3L 4 Cylinder Engine Gasoline Fuel,5-Speed M/T,Red,Black,None reported,Yes,"$6,300"
169,Ford,Escape Limited,2012,"129,000 mi.",Gasoline,171.0HP 2.5L 4 Cylinder Engine Gasoline Fuel,A/T,White,Gray,At least 1 accident or damage reported,Yes,"$7,000"
1689,Ford,F-150 Raptor,2022,"17,451 mi.",Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,Oxford White,Rhapsody Blue,None reported,NaN,"$77,900"
1791,Cadillac,Escalade ESV Premium Luxury,2021,"53,169 mi.",Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,A/T,Black,Black,None reported,Yes,"$81,500"


## Mising Values

In [230]:
df.isnull().sum()

brand             0
model             0
model_year        0
milage            0
fuel_type       215
engine           45
transmission      4
ext_col          15
int_col         133
accident        113
clean_title     596
price             0
dtype: int64

In [231]:
df['fuel_type'].unique()

array(['E85 Flex Fuel', 'Gasoline', 'Hybrid', nan, 'Diesel',
       'Plug-In Hybrid', 'not supported'], dtype=object)

In [232]:
df.groupby('fuel_type')['fuel_type'].count()

fuel_type
Diesel             116
E85 Flex Fuel      139
Gasoline          3309
Hybrid             194
Plug-In Hybrid      34
not supported        2
Name: fuel_type, dtype: int64

In [233]:
df['fuel_type'] = df['fuel_type'].fillna(df['fuel_type'].mode()[0])

In [234]:
df['brand'].nunique()

57

In [235]:
df.groupby('brand')['price'].mean().sort_values(ascending=False).head(10)

TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
df['accident'].unique()

array(['At least 1 accident or damage reported', 'None reported', nan],
      dtype=object)

In [236]:
df.groupby('accident')['accident'].count()

accident
At least 1 accident or damage reported     986
None reported                             2910
Name: accident, dtype: int64

In [237]:
df['accident'] = df['accident'].fillna(df['accident'].mode()[0])

In [238]:
accident_map = {'At least 1 accident or damage reported': 1, 'None reported': 0}
df['accident'] =df['accident'].apply(lambda x : accident_map[x])

In [239]:
df.groupby('clean_title')['clean_title'].count()

clean_title
Yes    3413
Name: clean_title, dtype: int64

In [240]:
df['clean_title'] = df['clean_title'].fillna('No')

In [241]:
df['brand'].unique()

array(['Ford', 'Hyundai', 'Lexus', 'INFINITI', 'Audi', 'Acura', 'BMW',
       'Tesla', 'Land', 'Aston', 'Toyota', 'Lincoln', 'Jaguar',
       'Mercedes-Benz', 'Dodge', 'Nissan', 'Genesis', 'Chevrolet', 'Kia',
       'Jeep', 'Bentley', 'Honda', 'Lucid', 'MINI', 'Porsche', 'Hummer',
       'Chrysler', 'Volvo', 'Cadillac', 'Lamborghini', 'Maserati',
       'Volkswagen', 'Subaru', 'Rivian', 'GMC', 'RAM', 'Alfa', 'Ferrari',
       'Scion', 'Mitsubishi', 'Mazda', 'Saturn', 'Bugatti', 'Polestar',
       'Rolls-Royce', 'McLaren', 'Buick', 'Lotus', 'Pontiac', 'FIAT',
       'Karma', 'Saab', 'Mercury', 'Plymouth', 'smart', 'Maybach',
       'Suzuki'], dtype=object)

In [242]:
df['transmission'].fillna(df['transmission'].mode()[0], inplace=True)
df['engine'].fillna(df['engine'].mode()[0], inplace=True)
df['ext_col'].fillna(df['ext_col'].mode()[0], inplace=True)
df['int_col'].fillna(df['int_col'].mode()[0], inplace=True)

In [243]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     4009 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      4009 non-null   int64 
 10  clean_title   4009 non-null   object
 11  price         4009 non-null   object
dtypes: int64(2), object(10)
memory usage: 376.0+ KB


# Feature Engineering

In [244]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     4009 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      4009 non-null   int64 
 10  clean_title   4009 non-null   object
 11  price         4009 non-null   object
dtypes: int64(2), object(10)
memory usage: 376.0+ KB


In [245]:
df.sample(5)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
3995,Lexus,CT 200h Base,2016,"39,750 mi.",Hybrid,134.0HP 1.8L 4 Cylinder Engine Gas/Electric Hy...,A/T,Black,Black,0,Yes,"$21,995"
308,Mercedes-Benz,CLK-Class 500 Cabriolet,2005,"64,000 mi.",Gasoline,302.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,Silver,Black,0,Yes,"$12,995"
2584,Chevrolet,Corvette Base,1998,"152,195 mi.",Gasoline,340.0HP 5.7L 8 Cylinder Engine Gasoline Fuel,A/T,Black,Black,0,Yes,"$9,900"
1335,Chevrolet,Suburban 1500 LTZ,2015,"217,500 mi.",Gasoline,355.0HP 5.3L 8 Cylinder Engine Gasoline Fuel,A/T,Black,Black,0,Yes,"$20,000"
2326,Porsche,911 R,2016,"12,454 mi.",Gasoline,500.0HP 4.0L Flat 6 Cylinder Engine Gasoline Fuel,6-Speed M/T,Purple,Black,0,Yes,"$210,000"


In [246]:
df['transmission_type'] = df['transmission'].apply(lambda x: 'Automatic' if 'a/t' in x.lower() or 'automatic' in x.lower() else 'Manual')

In [247]:
df['milage'] = df['milage'].str.replace(r'[^\d.]', '', regex=True).astype(float)
df['price'] = df['price'].str.replace(r'[^\d.]', '', regex=True).astype(float)

In [248]:
def extract_litre_eng(engine_desc):
    match = re.search(r'(\d+(\.\d+)?)\s*[Ll](?:iter)?', engine_desc)
    return float(match.group(1)) if match else None

df['engine_litre'] = df['engine'].apply(extract_litre_eng)
df['engine_litre'].fillna(df['engine_litre'].mode()[0], inplace=True)

In [249]:
def extract_hp(engine_desc):
    match  = re.search(r'(\d+(\.\d+)?)HP', engine_desc)
    return float(match.group(1)) if match else None

df_hp = pd.DataFrame()
df_hp['engine'] = df['engine']
df_hp['hp'] = df_hp['engine'].apply(extract_hp)
print('Hp Mean : ',df_hp['hp'].mean())
print('Hp Mode : ',df_hp['hp'].mode()[0])

Hp Mean :  332.20649796938454
Hp Mode :  300.0


In [250]:
df['hp'] = df['engine'].apply(extract_hp)
df['hp'].fillna(df['hp'].mode()[0], inplace=True)

In [251]:
def extract_cylinders(engine_desc):
    match = re.search(r'\b([VvIi]\d+)\b', engine_desc)
    if match:
        config = match.group(1)
        return int(config[1:])  

    match = re.search(r'(\d+(\.\d+)?)\s*L', engine_desc)
    if match:
        displacement = float(match.group(1))
        if displacement <= 2.5:
            return 4  
        elif displacement <= 3.8:
            return 6  
        else:
            return 8  
    return None


df['cylinders'] = df['engine'].apply(extract_cylinders)
df['cylinders'].fillna(df['cylinders'].mode()[0], inplace=True)

In [252]:
df['transmission'].unique()

array(['6-Speed A/T', '8-Speed Automatic', 'Automatic', '7-Speed A/T',
       'F', 'A/T', '8-Speed A/T', 'Transmission w/Dual Shift Mode',
       '9-Speed Automatic', '6-Speed M/T', '10-Speed A/T', '9-Speed A/T',
       '5-Speed A/T', '1-Speed A/T', 'Automatic CVT',
       '7-Speed Automatic with Auto-Shift', 'CVT-F', 'M/T',
       '6-Speed Automatic with Auto-Shift', '10-Speed Automatic',
       'CVT Transmission', '4-Speed A/T', '6-Speed Automatic',
       '4-Speed Automatic', '7-Speed M/T',
       '8-Speed Automatic with Auto-Shift', '5-Speed Automatic',
       '8-SPEED AT', '1-Speed Automatic', '5-Speed M/T', 'Manual',
       '6-Speed Manual', 'Variable', '2', '7-Speed Manual',
       'Automatic, 9-Spd 9G-Tronic', 'Automatic, 8-Spd',
       'Automatic, 8-Spd Sport w/Sport & Manual Modes',
       'Auto, 6-Spd w/CmdShft', 'Transmission Overdrive Switch',
       '7-Speed Automatic', 'Automatic, 8-Spd PDK Dual-Clutch',
       'Automatic, 8-Spd M STEPTRONIC w/Drivelogic, Sport & Manual 

In [253]:
def extract_speed(transmission_desc):
    match = re.search(r'(\d+)[- ]?(Speed|Spd)', transmission_desc, re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        return None

extract_speed('6-Speed A/T')

'6'

In [255]:
df['transmission_speed'] = df['transmission'].apply(extract_speed)

In [256]:
df[df['transmission_speed'].isnull()]['transmission'].unique()

array(['Automatic', 'F', 'A/T', 'Transmission w/Dual Shift Mode',
       'Automatic CVT', 'CVT-F', 'M/T', 'CVT Transmission', 'Manual',
       'Variable', '2', 'Transmission Overdrive Switch',
       'Single-Speed Fixed Gear', 'SCHEDULED FOR OR IN PRODUCTION'],
      dtype=object)

In [257]:
df.loc[df['transmission'] == '2', 'transmission_speed'] = '2'
df.loc[df['transmission'] == 'Single-Speed Fixed Gear', 'transmission_speed'] = '1'

In [258]:
df['transmission_speed'].fillna('5', inplace= True)

In [259]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   brand               4009 non-null   object 
 1   model               4009 non-null   object 
 2   model_year          4009 non-null   int64  
 3   milage              4009 non-null   float64
 4   fuel_type           4009 non-null   object 
 5   engine              4009 non-null   object 
 6   transmission        4009 non-null   object 
 7   ext_col             4009 non-null   object 
 8   int_col             4009 non-null   object 
 9   accident            4009 non-null   int64  
 10  clean_title         4009 non-null   object 
 11  price               4009 non-null   float64
 12  transmission_type   4009 non-null   object 
 13  engine_litre        4009 non-null   float64
 14  hp                  4009 non-null   float64
 15  cylinders           4009 non-null   float64
 16  transm

In [260]:
df.sample(5)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,transmission_type,engine_litre,hp,cylinders,transmission_speed
2659,Mercedes-Benz,S-Class S 580 4MATIC,2023,989.0,Gasoline,4.0L V8 32V GDI DOHC Twin Turbo,9-Speed Automatic,Black,Black,0,Yes,122950.0,Automatic,4.0,300.0,8.0,9
2351,Land,Rover Range Rover Sport P400 SE Dynamic,2023,450.0,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Blue,Black,0,Yes,124000.0,Automatic,3.0,395.0,6.0,8
2284,BMW,330 i,2017,83298.0,Gasoline,248.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Black,Black,0,Yes,18300.0,Automatic,2.0,248.0,4.0,5
398,BMW,Z4 sDrive28i,2015,57550.0,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed M/T,Black,White,0,Yes,29995.0,Manual,2.0,240.0,4.0,6
420,McLaren,570S Spider,2020,8700.0,Gasoline,562.0HP 3.8L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,0,Yes,185000.0,Manual,3.8,562.0,6.0,5


In [261]:
cols_drop = ['engine','transmission', 'model' ]
df.drop(columns=cols_drop,axis=1, inplace=True)

In [262]:
clean_title_map = {'Yes': 1, 'No': 0}
df['clean_title'] = df['clean_title'].apply(lambda x : clean_title_map[x])

In [263]:
df.sample(5)

,brand,model_year,milage,fuel_type,ext_col,int_col,accident,clean_title,price,transmission_type,engine_litre,hp,cylinders,transmission_speed
1689,Ford,2022,17451.0,Gasoline,Oxford White,Rhapsody Blue,0,0,77900.0,Automatic,3.5,300.0,6.0,10
3832,Jeep,2005,162482.0,Gasoline,Orange,Gray,0,1,11000.0,Manual,4.0,190.0,8.0,6
3297,Mercedes-Benz,2019,40000.0,Gasoline,Blue,Black,0,1,27900.0,Automatic,2.0,255.0,4.0,5
984,Jeep,2022,12875.0,Gasoline,Diamond Black,Global Black,0,0,42599.0,Automatic,3.6,300.0,6.0,8
373,Audi,2015,126760.0,Gasoline,Black,Black,1,1,14000.0,Automatic,2.0,211.0,4.0,5


# Encoding

In [264]:
cols_encode = ['brand', 'fuel_type', 'transmission_type', 'ext_col', 'int_col', 'model_year']
cols_scale = ['milage' , 'hp', 'cylinders', 'transmission_speed', 'engine_litre']

In [265]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df[cols_encode] = df[cols_encode].apply(le.fit_transform)

In [266]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df[cols_scale] = sc.fit_transform(df[cols_scale])

In [267]:
df.sample(5)

,brand,model_year,milage,fuel_type,ext_col,int_col,accident,clean_title,price,transmission_type,engine_litre,hp,cylinders,transmission_speed
3676,15,24,0.598248,2,38,10,0,1,17000.0,0,0.004701,-0.406541,-0.082046,0.003315
1721,14,28,-0.420530,2,29,31,0,1,45950.0,1,-0.065223,1.129963,-0.082046,-0.600719
3690,15,26,1.133723,2,128,10,1,1,32000.0,0,1.822720,0.857211,1.190388,1.211382
2465,15,26,0.999854,1,263,138,0,1,38000.0,0,1.682873,0.148055,1.190388,0.003315
2747,8,25,0.349635,2,234,10,0,1,16500.0,0,-1.114081,-0.488366,-1.354481,1.211382


# Model Train

In [268]:
x = df.drop(columns=['price'])
y = df['price']

In [269]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [270]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

RandomForestRegressor(random_state=42)

In [274]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
dtr.fit(x_train, y_train)

from sklearn.metrics import mean_absolute_error, mean_squared_error
y_pred = dtr.predict(x_test)
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

MAE: 23814.50374064838
MSE: 19426029091.2793
RMSE: 139377.2904431683


In [275]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()
knn.fit(x_train, y_train)

from sklearn.metrics import mean_absolute_error, mean_squared_error
y_pred = knn.predict(x_test)
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

MAE: 26629.84788029925
MSE: 19931418320.98703
RMSE: 141178.67516373366


In [278]:
knn_pipeline = knn.fit(x_train, y_train)

from sklearn.model_selection import GridSearchCV

param_grid = {
    'model__n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'model__weights': ['uniform', 'distance'],
    'model__metric': ['euclidean', 'manhattan']
}

# Grid Search
grid_search = GridSearchCV(knn_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(x_train, y_train)


# Best model
best_knn_model = grid_search.best_estimator_

# Predict and evaluate
y_pred_best_knn = best_knn_model.predict(x_test)
rmse_best_knn = mean_squared_error(y_test, y_pred_best_knn, squared=False)

print(f'Best KNN Regression RMSE: {rmse_best_knn}')
print(f'Best Parameters: {grid_search.best_params()}')


ValueError: Invalid parameter 'model' for estimator KNeighborsRegressor(). Valid parameters are: ['algorithm', 'leaf_size', 'metric', 'metric_params', 'n_jobs', 'n_neighbors', 'p', 'weights'].

In [271]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
y_pred = model.predict(x_test)
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

MAE: 18953.81624688279
MSE: 18015162635.88779
RMSE: 134220.57456250064


In [ ]:
import joblib
joblib.dump(model, 'car_price_model.pkl')